In [1]:
import requests
import json
import datetime

In [2]:
def r_to_j(r):
    myJ_s = ''
    line_num=1
    for n in r:
        #print(line_num, n)
        myJ_s += n.decode("'iso-8859-1'")
        
#    print(myJ_s)
    myJ = json.loads(myJ_s)
#    print(json.dumps(myJ_s, indent=1))
    return myJ


In [3]:
def do_r(req, page_size=2000, page_num=0,debug=False, attrs=[]):
    myJ = {}
    rJ = {'count': 0, 'g' : {}}
    keepGoing=1

    req += '&page_size='+str(page_size)+'&page_num='

    while keepGoing:
    
        page_num += 1
        if page_num > 499:
            keepGoing = 0
        
        r = requests.get(req + str(page_num) )
        if debug:
            print(r, page_num, datetime.datetime.now())
        
        myJ = r_to_j(r)
        #print('myj=', json.dumps(myJ, indent=1))
        if len(myJ['feed']['entry']) == 0:
            keepGoing=0
            continue

        rJ['count']+= len(myJ['feed']['entry'])
        outJ={}
        for j in myJ['feed']['entry']:
            outJ={}
            for a in attrs:
                #print(a,'a=')
                if a in j:
                    outJ[a]=j[a]
                
            rJ['g'][j['id']]=outJ

    return rJ

In [4]:
# Get a list of short_names from a provider

my_provider='LAADS'
my_shortNames=[]
#print (r.headers)
myJ_s = ''
collectionsJ={}
uniqueKeysJ={}

page_num=0
keepGoing=1

while keepGoing:
    
    page_num += 1
    if page_num > 200:
        keepGoing = 0
        
    #r = requests.get('https://cmr.earthdata.nasa.gov/search/collections.umm_json?platform=AQUA&sort_key=short_name&page_size=2&page_num='+str(page_num))
    #r = requests.get('https://cmr.earthdata.nasa.gov/search/collections.umm_json?provider=GES_DISC&sort_key=short_name&page_size=2000&page_num='+str(page_num))
    #r = requests.get('https://cmr.earthdata.nasa.gov/search/collections.umm_json?instrument=MODIS&INSTRUMENT=MODIS&provider=LPDAAC_ECS&sort_key=short_name&page_size=2&page_num='+str(page_num))
    r = requests.get('https://cmr.earthdata.nasa.gov/search/collections.umm_json?&provider=LAADS&sort_key=short_name&page_size=2000&page_num='+str(page_num))

    myJ_s = ''
    line_num=1

    for n in r:
        #print(line_num, n)
        myJ_s += n.decode("'iso-8859-1'")
        
#    print(myJ_s)
    myJ = json.loads(myJ_s)
    #print(json.dumps(myJ, indent=1))

    if len(myJ['items']) == 0:
        keepGoing=0
        continue
    
    printIt=0
    for r in myJ['items']:
        j=r['meta']
        #print ('j=', j)
        if printIt:
            print ('r=', json.dumps(r,indent=1))
            printIt=0
        #for k in j:
        #    print ('meta',k, j[k])
        j=r['umm']
#        for k in j:
            #print ('umm', k)
#            if k == 'ShortName':
        my_shortNames.append({
            'ShortName' : j['ShortName'], 
            'native-id' : r['meta']['native-id'],
            'EntryTitle' : j['EntryTitle'],
            'deleted' : r['meta']['deleted'],
            'Version': j['Version'],
            'ProcessingLevel' : j['ProcessingLevel']
        })

i=1
for s in my_shortNames:
    print (i, s)
    i+=1
#print (my_shortNames)

1 {'ShortName': 'AERDB_D3_VIIRS_SNPP', 'native-id': 'LAADS_AERDB_D3_VIIRS_SNPP_1', 'EntryTitle': 'VIIRS/SNPP Deep Blue Level 3 daily aerosol data, 1x1 degree grid', 'deleted': False, 'Version': '1', 'ProcessingLevel': {'Id': '3'}}
2 {'ShortName': 'AERDB_L2_VIIRS_SNPP', 'native-id': 'LAADS_AERDB_L2_VIIRS_SNPP_1', 'EntryTitle': 'VIIRS/SNPP Deep Blue Aerosol L2 6-Min Swath 6 km', 'deleted': False, 'Version': '1', 'ProcessingLevel': {'Id': '2'}}
3 {'ShortName': 'AERDB_M3_VIIRS_SNPP', 'native-id': 'LAADS_AERDB_M3_VIIRS_SNPP_1', 'EntryTitle': 'VIIRS/SNPP Deep Blue Level 3 monthly aerosol data, 1x1 degree grid', 'deleted': False, 'Version': '1', 'ProcessingLevel': {'Id': '3'}}
4 {'ShortName': 'AERDT_L2_VIIRS_SNPP', 'native-id': 'LAADS_AERDT_L2_VIIRS_SNPP_1', 'EntryTitle': 'VIIRS/SNPP Dark Target Aerosol L2 6-Min Swath 6 km', 'deleted': False, 'Version': '1.1', 'ProcessingLevel': {'Id': '2'}}
5 {'ShortName': 'CLDCR_L2_VIIRS_SNPP', 'native-id': 'mmt_collection_12770', 'EntryTitle': 'VIIRS/SNPP 

In [ ]:
r = 'https://cmr.earthdata.nasa.gov/search/granules.json?provider_id=LAADS&sort_key=start_date&short_name=$1&version=$2&temporal=2020-01-01T00:00:00Z,'
i=1

my_shortNames=[
    {'short_name': 'MYD01', 'version' : '6.1'   , 'g' : {}},
    {'short_name': 'MYD03', 'version' : '6.1'   , 'g' : {}},
    {'short_name': 'MYD021KM', 'version' : '6.1', 'g' : {} },
    {'short_name': 'MYD02HKM', 'version' : '6.1', 'g' : {} },
    {'short_name': 'MYD02QKM', 'version' : '6.1', 'g' : {} }    
]

# MYD Start Date start_date=2002-07-04T00:00:00.000Z

for s in my_shortNames:
    #print (i, s)
    i+=1

    r = r.replace('$1', s['short_name'])
    r = r.replace('$2', s['version'])

    gJ = do_r(r, attrs=['short_name','version','time_start', 'time_end'],debug=True)
    print ('# Granules', gJ['count'])
    
    s['g']=gJ['g']
    
for s in my_shortNames:
    for g in s['g']:
        print (g, s['g'][g])
        break
    



<Response [200]> 1 2021-05-06 16:23:58.089334
<Response [200]> 2 2021-05-06 16:23:58.777385
<Response [200]> 3 2021-05-06 16:23:59.584363
<Response [200]> 4 2021-05-06 16:24:00.263119
<Response [200]> 5 2021-05-06 16:24:01.210902
<Response [200]> 6 2021-05-06 16:24:01.914571
<Response [200]> 7 2021-05-06 16:24:02.716940
<Response [200]> 8 2021-05-06 16:24:04.123506
<Response [200]> 9 2021-05-06 16:24:04.917885
<Response [200]> 10 2021-05-06 16:24:05.632447
<Response [200]> 11 2021-05-06 16:24:06.585741
<Response [200]> 12 2021-05-06 16:24:07.268264
<Response [200]> 13 2021-05-06 16:24:08.042894
<Response [200]> 14 2021-05-06 16:24:09.025945
<Response [200]> 15 2021-05-06 16:24:09.866140
<Response [200]> 16 2021-05-06 16:24:10.688941
<Response [200]> 17 2021-05-06 16:24:11.421152
<Response [200]> 18 2021-05-06 16:24:12.182497
<Response [200]> 19 2021-05-06 16:24:13.003694
<Response [200]> 20 2021-05-06 16:24:13.805960
<Response [200]> 21 2021-05-06 16:24:14.492053
<Response [200]> 22 20

In [ ]:
#https://modaps.modaps.eosdis.nasa.gov/services/production/outages_aqua.html
gaps={}

aquaDataOutages_str=[
    ['2020-01-20T07:55:00.000Z', '2020-01-29T08:00:00.000Z'],
    ['2020-03-05T16:00:00.000Z', '2020-03-05T16:05:00.000Z'],
    ['2020-03-12T16:10:00.000Z', '2020-03-12T16:15:00.000Z'],
    ['2020-03-26T16:20:00.000Z', '2020-03-26T16:25:00.000Z'],
    ['2020-03-30T22:00:00.000Z', '2020-03-30T23:40:00.000Z'],
    ['2020-04-02T16:30:00.000Z', '2020-04-02T16:35:00.000Z'],
    [ '2020-05-11T09:30:00.000Z', '2020-05-11T10:00:00.000Z'],
    [ '2020-09-10T15:30:00.000Z', '2020-09-10T15:35:00.000Z'],
    ['2020-09-17T15:40:00.000Z', '2020-09-17T15:45:00.000Z'],
    ['2020-09-26T08:10:00.000Z', '2020-09-26T08:15:00.000Z']
]

aquaDataOutages = []
for o in aquaDataOutages_str:
    datetime.datetime.strptime('2020-01-20T07:55:00.000Z','%Y-%m-%dT%H:%M:%S.%fZ')
    aquaDataOutages.append([datetime.datetime.strptime(o[0],'%Y-%m-%dT%H:%M:%S.%fZ'),datetime.datetime.strptime(o[1],'%Y-%m-%dT%H:%M:%S.%fZ')])

#print (aquaDataOutages)
for s in my_shortNames:
    i=0
    #print ("Gaps for ", s['short_name'], s['version'])
    for k in sorted(s['g'].items(), key=lambda x: x[1]['time_start']):
        if i > 0:
            if not prev_k['time_end'] == k[1]['time_start']:
                #print ('Gap', prev_k['time_end'], '<----->', k[1]['time_start'])
                g_start=datetime.datetime.strptime(prev_k['time_end'],'%Y-%m-%dT%H:%M:%S.%fZ')
                g_end  = datetime.datetime.strptime(k[1]['time_start'], '%Y-%m-%dT%H:%M:%S.%fZ')
                outageOverlap=False
                for r in aquaDataOutages:
                    #print(r)
                    if g_start <= r[1] and g_end >= r[0]:
                        #print('Data Outage Overlap', r[0], r[1])
                        outageOverlap=True
                        break
                if outageOverlap:
                    pass
                else:
                    gapKey=prev_k['time_end']+"~"+k[1]['time_start']
                    if gapKey in gaps:
                        gaps[gapKey].append({'short_name' : s['short_name'], 'version': s['version']})
                    else:
                        gaps[gapKey]=[{'short_name' : s['short_name'], 'version': s['version']}]
                
                #print (i, prev_k, k)
        prev_k = k[1]
        i+=1

for g in gaps:
    print (g.replace('T', ' '), end=": ")
    #print (json.dumps(gaps[g]))
    print (' ShortNames/Versions:', end='' )
    for x in gaps[g]:
        print (x['short_name'] + ":" + x['version'], end=',')
    print()


In [ ]:
i=0
#{1: 2, 3: 4, 4: 3, 2: 1, 0: 0}
#{ k: v for k, v in sorted(x.items(), key=lambda item: item[1])}
for s in my_shortNames:
    i=0
    print(len(s['g']))
    for k in sorted(s['g'].items(), key=lambda x: x[1]['time_start']):
        if i > 5780:
            print (i,k)
        i+=1
 